<a href="https://colab.research.google.com/github/Zubyloops/zindi/blob/master/Classification_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Honour Code
I Zubair, Samsodien confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [0]:
"""Import Builtin Python Modules"""
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='notebook', palette='viridis')
sns.despine(top=True, right=True)

import datetime
import re
import io

%matplotlib inline

from imblearn.over_sampling import SMOTE

"""Import SciKit Learn Modules"""
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler ,LabelEncoder
from sklearn.linear_model import LogisticRegression,SGDClassifier

from sklearn.metrics import classification_report, accuracy_score, log_loss, confusion_matrix

# Data import

from google.colab import files
uploaded = files.upload()

In [0]:
training = pd.read_csv('training.csv')
testing = pd.read_csv('test.csv')

In [0]:
train= training.copy()
test = testing.copy()

In [0]:
train.shape,test.shape

((95662, 16), (45019, 15))

In [0]:
df = pd.concat([train, test], axis = 0, sort=False)

# Data Cleaning

In [0]:
"""Python
col = ["TransactionId","BatchId","AccountId","SubscriptionId","CustomerId","ProviderId","ProductId","ChannelId"]
df[col] = df[col].replace(to_replace = r'\D+', value = '', regex = True)
#df["TransactionStartTime"] = df["TransactionStartTime"].replace(to_replace = r'\D+', value = ' ', regex = True)

df["TransactionStartTime"] = pd.to_date[link text](https://)time(df["TransactionStartTime"])
df["TransactionStartTime"]= df["TransactionStartTime"].values.astype('datetime64[s]')
df['TransactionStartTime'] = df['TransactionStartTime'].astype(object)

id_col =["TransactionId","BatchId","AccountId","SubscriptionId","SubscriptionId"]
df[id_col]= df[id_col].values.astype(int)"""

'Python\ncol = ["TransactionId","BatchId","AccountId","SubscriptionId","CustomerId","ProviderId","ProductId","ChannelId"]\ndf[col] = df[col].replace(to_replace = r\'\\D+\', value = \'\', regex = True)\n#df["TransactionStartTime"] = df["TransactionStartTime"].replace(to_replace = r\'\\D+\', value = \' \', regex = True)\n\ndf["TransactionStartTime"] = pd.to_date[link text](https://)time(df["TransactionStartTime"])\ndf["TransactionStartTime"]= df["TransactionStartTime"].values.astype(\'datetime64[s]\')\ndf[\'TransactionStartTime\'] = df[\'TransactionStartTime\'].astype(object)\n\nid_col =["TransactionId","BatchId","AccountId","SubscriptionId","SubscriptionId"]\ndf[id_col]= df[id_col].values.astype(int)'

df['diff_seconds'] = df['TransactionStartTime'] - df['TransactionStartTime'].shift(1)
df['diff_seconds']=df['diff_seconds']/np.timedelta64(1,'s')


df['diff_seconds'].iloc[0] = 0

# Encoding

In [0]:

sc = StandardScaler()
df['normAmount'] = sc.fit_transform(df['Amount'].values.reshape(-1, 1))
df['normValue'] = sc.fit_transform(df['Value'].values.reshape(-1, 1))


In [0]:
df.drop(["CurrencyCode","CountryCode",'Amount','Value',"TransactionStartTime"],axis=1,inplace= True)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 13 columns):
TransactionId      140681 non-null int64
BatchId            140681 non-null object
AccountId          140681 non-null object
SubscriptionId     140681 non-null object
CustomerId         140681 non-null object
ProviderId         140681 non-null object
ProductId          140681 non-null object
ProductCategory    140681 non-null object
ChannelId          140681 non-null object
PricingStrategy    140681 non-null int64
FraudResult        95662 non-null float64
normAmount         140681 non-null float64
normValue          140681 non-null float64
dtypes: float64(3), int64(2), object(8)
memory usage: 15.0+ MB


In [0]:
le = LabelEncoder()
df['ProductCategory'] = le.fit_transform(df.ProductCategory.values)

In [0]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,PricingStrategy,FraudResult,normAmount,normValue
0,76871,36123,3957,887,4406,6,10,0,3,2,0.0,-0.043799,-0.078466
1,73770,15642,4841,3829,4406,4,6,2,2,2,0.0,-0.052644,-0.086983
2,26203,53941,4229,222,4683,6,1,0,3,2,0.0,-0.048135,-0.082812
3,380,102363,648,2185,988,1,21,9,3,2,0.0,0.120970,0.102305
4,28195,38780,4841,3829,988,4,6,2,2,2,0.0,-0.058056,-0.081560


In [0]:
df1 = pd.get_dummies(df,drop_first=True,columns =["ProductCategory","ProviderId","ProductId","ChannelId","PricingStrategy"])

In [0]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 55 columns):
TransactionId        140681 non-null int64
BatchId              140681 non-null object
AccountId            140681 non-null object
SubscriptionId       140681 non-null object
CustomerId           140681 non-null object
FraudResult          95662 non-null float64
normAmount           140681 non-null float64
normValue            140681 non-null float64
ProductCategory_1    140681 non-null uint8
ProductCategory_2    140681 non-null uint8
ProductCategory_3    140681 non-null uint8
ProductCategory_4    140681 non-null uint8
ProductCategory_5    140681 non-null uint8
ProductCategory_6    140681 non-null uint8
ProductCategory_7    140681 non-null uint8
ProductCategory_8    140681 non-null uint8
ProductCategory_9    140681 non-null uint8
ProviderId_2         140681 non-null uint8
ProviderId_3         140681 non-null uint8
ProviderId_4         140681 non-null uint8
ProviderId_5         

print(len(df.AccountId.unique()))

print(len(df.CustomerId.unique()))
print(len(df.SubscriptionId.unique()))
print(len(df.BatchId.unique()))
print(len(df.TransactionId.unique()))

In [0]:
df1.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,FraudResult,diff_seconds,normAmount,normValue,ProductCategory_1,ProductCategory_2,ProductCategory_3,ProductCategory_4,ProductCategory_5,ProductCategory_6,ProductCategory_7,ProductCategory_8,ProductCategory_9,ProviderId_2,ProviderId_3,ProviderId_4,ProviderId_5,ProviderId_6,ProductId_10,ProductId_11,ProductId_12,ProductId_13,ProductId_14,ProductId_15,ProductId_16,ProductId_17,ProductId_18,ProductId_19,ProductId_2,ProductId_20,ProductId_21,ProductId_22,ProductId_23,ProductId_24,ProductId_25,ProductId_26,ProductId_27,ProductId_3,ProductId_4,ProductId_5,ProductId_6,ProductId_7,ProductId_8,ProductId_9,ChannelId_2,ChannelId_3,ChannelId_4,ChannelId_5,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4
0,76871,36123,3957,887,4406,0.0,0.0,-0.043799,-0.078466,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,73770,15642,4841,3829,4406,0.0,19.0,-0.052644,-0.086983,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
2,26203,53941,4229,222,4683,0.0,1513.0,-0.048135,-0.082812,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,380,102363,648,2185,988,0.0,2914.0,0.120970,0.102305,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,28195,38780,4841,3829,988,0.0,86.0,-0.058056,-0.081560,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0


# Train/Test Split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
X = df1.drop(columns=["FraudResult"])
y = train["FraudResult"]

In [0]:
X.shape,y.shape

((140681, 55), (95662,))

In [0]:
train.shape,test.shape,df1.shape

((95662, 16), (45019, 15), (140681, 56))

In [0]:
X_trainn = X[:95662].copy()
X_testt = X[95662:].copy()

In [0]:
X_trainn.shape,y.shape

((95662, 55), (95662,))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_trainn, y, test_size = 0.2, random_state = 0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (76529, 55)
Number transactions y_train dataset:  (76529,)
Number transactions X_test dataset:  (19133, 55)
Number transactions y_test dataset:  (19133,)


In [0]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 158
Before OverSampling, counts of label '0': 76371 

After OverSampling, the shape of train_X: (152742, 55)
After OverSampling, the shape of train_y: (152742,) 

After OverSampling, counts of label '1': 76371
After OverSampling, counts of label '0': 76371


# ML Model Training

In [0]:
logR = LogisticRegression()

logR = logR.fit(X_train, y_train)

y_pred_train = logR.predict(X_test)
print(accuracy_score(y_test, y_pred_train))

print('The accuracy score of the SVC is:', accuracy_score(y_test, y_pred_train))
print('\nThe test confusion matrix:\n', confusion_matrix(y_test,y_pred_train))
print('\n\nClassification Report:\n\n', classification_report(y_test, y_pred_train))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.998484294151466
The accuracy score of the SVC is: 0.998484294151466

The test confusion matrix:
 [[19092     6]
 [   23    12]]


Classification Report:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00     19098
           1       0.67      0.34      0.45        35

    accuracy                           1.00     19133
   macro avg       0.83      0.67      0.73     19133
weighted avg       1.00      1.00      1.00     19133



In [0]:
lr1 = LogisticRegression()
y_pred_train = lr1.fit(X_train_res, y_train_res.ravel())

a = y_pred_train.predict(X_test)

print(accuracy_score(y_test, a))
#print('The accuracy score of the LR is:', accuracy_score(y_test, y_pred_train))
#print('\nThe test confusion matrix:\n', confusion_matrix(y_test,y_pred_train))
print('\n\nClassification Report:\n\n', classification_report(y_test, a))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9967595254272722


Classification Report:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00     19098
           1       0.35      0.94      0.52        35

    accuracy                           1.00     19133
   macro avg       0.68      0.97      0.76     19133
weighted avg       1.00      1.00      1.00     19133



# Submission

a = y_pred_train.predict(X_test)

In [0]:
predictions = logR.predict(X_testt)

predictions = y_pred_train.predict(X_testt)

In [0]:
testing['FraudResult'] =  predictions


testing['FraudResult'] = y_pred_train.predict(X_testt)

In [0]:
testing = testing[["TransactionId",'FraudResult']]

In [0]:
testing

,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0
5,TransactionId_134338,0
6,TransactionId_109096,0
7,TransactionId_14249,0
8,TransactionId_69896,0
9,TransactionId_91468,0


In [0]:
"""Export the resulting submission"""
testing.to_csv(r'Submission_6.csv', index = None, header=True)

from google.colab import files
files.download('Submission_6.csv') 